# Ortalama Kayma (MeanShift) Algoritması

In [1]:
import cv2
import numpy as np

In [2]:
cap=cv2.VideoCapture(0)
ret,frame=cap.read()
if ret==False:
    print("okuma hatasi")
face_cascade=cv2.CascadeClassifier("input/Haar cascade/4.1 frontalface.xml.xml")
face_rects=face_cascade.detectMultiScale(frame)

# buraya kadarı benim "GoruntuIsleme/1.Egitim/14." de yaptığım gibi ama buradan sonra bu eğitimde farklı yol izliyor
(face_x,face_y,w,h)=tuple(face_rects[0])
track_winodw=(face_x,face_y,w,h) # MeanShiftalgoritması girdisi
# region of interest
roi=frame[face_y:face_y+h,face_x:face_x+w] # roi=face
hsv_roi=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180]) # takip için histogram gerekli
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# takip için gerekli durdurma kriterleri
# count = hesaplanacak gerekli öge sayısı
# eps=değişiklik
term_crit=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,5,1)

while True:
    ret, frame = cap.read()
    # frame=cv2.flip(frame,1) bunu uygulayınca takip edemedi
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # histogramı bir görüntüde bulmak için kullanıyoruz
        # piksel karşılaştırma
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)


        ret,track_winodw=cv2.meanShift(dst,track_winodw,term_crit)
        x,y,w,h=track_winodw
        img2=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)
        cv2.imshow("yuz",img2)
        if cv2.waitKey(1)&0xff==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ibrahim/.local/lib/python3.10/site-packages/cv2/qt/plugins"


## Tüm kod ayrıntılı açıklama :
Elbette, bu kod parçası, bir video akışındaki yüzü tespit etmek ve MeanShift algoritması kullanarak yüzü takip etmek için OpenCV kütüphanesini kullanmaktadır. Kodun her bir parçasını ayrıntılı olarak açıklayacağım:

### Kodun İlk Kısmı: Yüz Tespiti

```python
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
if ret == False:
    print("okuma hatasi")
```

- `cap = cv2.VideoCapture(0)`: Bilgisayarınızın varsayılan web kamerasını (cihaz indeksi 0) kullanarak bir video yakalama nesnesi oluşturur.
- `ret, frame = cap.read()`: Kameradan bir kare (frame) okur. `ret` değeri, okumanın başarılı olup olmadığını belirtir. `frame` ise okunan görüntüdür.
- `if ret == False:`: Eğer kare okunamazsa, "okuma hatasi" mesajını yazdırır.

```python
face_cascade = cv2.CascadeClassifier("input/Haar cascade/4.1 frontalface.xml.xml")
face_rects = face_cascade.detectMultiScale(frame)
```

- `face_cascade = cv2.CascadeClassifier("input/Haar cascade/4.1 frontalface.xml.xml")`: Haar yüz tanıma kaskadını yükler. Bu dosya, önceden eğitilmiş bir yüz algılama modelini içerir.
- `face_rects = face_cascade.detectMultiScale(frame)`: Yüzleri `frame` içinde tespit eder. Bu fonksiyon, tespit edilen yüzlerin koordinatlarını içeren bir liste döner.

### Kodun İkinci Kısmı: MeanShift Algoritması İçin Hazırlık

```python
(face_x, face_y, w, h) = tuple(face_rects[0])
track_winodw = (face_x, face_y, w, h)  # MeanShift algoritması girdisi
```

- `(face_x, face_y, w, h) = tuple(face_rects[0])`: Tespit edilen ilk yüzün koordinatlarını alır.
- `track_winodw = (face_x, face_y, w, h)`: MeanShift algoritması için izleme penceresini tanımlar.

```python
roi = frame[face_y:face_y+h, face_x:face_x+w]  # roi=face
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])  # takip için histogram gerekli
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
```

- `roi = frame[face_y:face_y+h, face_x:face_x+w]`: Yüz bölgesini (Region of Interest - ROI) seçer.
- `hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)`: ROI'yi HSV renk uzayına dönüştürür. HSV (Hue, Saturation, Value) renk uzayı, renk takibi için daha uygundur.
- `roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])`: HSV ROI'nin histogramını hesaplar. Bu histogram, takip sırasında kullanılır.
- `cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)`: Histogramı normalleştirir. Histogram değerlerini 0 ile 255 arasında ölçeklendirir.

### Kodun Üçüncü Kısmı: İzleme

```python
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1)
```

- `term_crit`: MeanShift algoritması için durdurma kriterlerini tanımlar. `cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT` ile her iki kriter de kullanılacaktır: belirli bir sayıdaki iterasyon (`count`) ve pozisyon değişim miktarı (`eps`).

```python
while True:
    ret, frame = cap.read()
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        ret, track_winodw = cv2.meanShift(dst, track_winodw, term_crit)
        x, y, w, h = track_winodw
        img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 5)
        cv2.imshow("yuz", img2)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
```

- `while True`: Sonsuz bir döngü başlatır.
- `ret, frame = cap.read()`: Kameradan bir kare okur.
- `if ret:`: Eğer kare okunmuşsa devam eder.
    - `hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)`: Mevcut kareyi HSV renk uzayına dönüştürür.
    - `dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)`: Mevcut karenin histogramına dayanarak, ROI'nin olası konumlarını belirler.
    - `ret, track_winodw = cv2.meanShift(dst, track_winodw, term_crit)`: MeanShift algoritmasını kullanarak yeni izleme penceresini hesaplar.
    - `x, y, w, h = track_winodw`: Yeni izleme penceresinin koordinatlarını alır.
    - `img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 5)`: İzleme penceresini kırmızı bir dikdörtgenle çizer.
    - `cv2.imshow("yuz", img2)`: Çerçeveyi ekranda gösterir.
    - `if cv2.waitKey(1) & 0xff == ord('q')`: 'q' tuşuna basıldığında döngüyü kırar.
- `cap.release()`: Kamerayı serbest bırakır.
- `cv2.destroyAllWindows()`: Tüm OpenCV pencerelerini kapatır.


# Keşisfsel Veri analizi

In [4]:
import cv2
import os
from os.path import isfile , join
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import seaborn as sns

In [5]:
def write_to_file(bound_box_list, output_path):
    with open(output_path, 'w') as file:
        file.write("frame_no,x,y,w,h,center_x,center_y\n")
        for item in bound_box_list:
            file.write(','.join(map(str, item)) + '\n')

In [3]:
# Define input and output paths
pathIn = r"input/img1"
pathOut = "input/deneme.mp4"

# Get sorted list of files
files = sorted([f for f in os.listdir(pathIn) if isfile(join(pathIn, f))])

fps = 25
size = (1920, 1080)
out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*"MP4V"), fps, size, True)

for i in files:
    print(i)
    filename = join(pathIn, i)
    img = cv2.imread(filename)
    
    # Check if the image is read correctly
    if img is None:
        print(f"Failed to read {filename}")
        continue
    
    # Resize the image to match the video size
    img = cv2.resize(img, size)
    
    out.write(img)

out.release()


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


000001.jpg
000002.jpg
000003.jpg
000004.jpg
000005.jpg
000006.jpg
000007.jpg
000008.jpg
000009.jpg
000010.jpg
000011.jpg
000012.jpg
000013.jpg
000014.jpg
000015.jpg
000016.jpg
000017.jpg
000018.jpg
000019.jpg
000020.jpg
000021.jpg
000022.jpg
000023.jpg
000024.jpg
000025.jpg
000026.jpg
000027.jpg
000028.jpg
000029.jpg
000030.jpg
000031.jpg
000032.jpg
000033.jpg
000034.jpg
000035.jpg
000036.jpg
000037.jpg
000038.jpg
000039.jpg
000040.jpg
000041.jpg
000042.jpg
000043.jpg
000044.jpg
000045.jpg
000046.jpg
000047.jpg
000048.jpg
000049.jpg
000050.jpg
000051.jpg
000052.jpg
000053.jpg
000054.jpg
000055.jpg
000056.jpg
000057.jpg
000058.jpg
000059.jpg
000060.jpg
000061.jpg
000062.jpg
000063.jpg
000064.jpg
000065.jpg
000066.jpg
000067.jpg
000068.jpg
000069.jpg
000070.jpg
000071.jpg
000072.jpg
000073.jpg
000074.jpg
000075.jpg
000076.jpg
000077.jpg
000078.jpg
000079.jpg
000080.jpg
000081.jpg
000082.jpg
000083.jpg
000084.jpg
000085.jpg
000086.jpg
000087.jpg
000088.jpg
000089.jpg
000090.jpg
000091.jpg

In [6]:
pathOut = "input/deneme.mp4"
# şimdi de analiz aşaması
col_list=["frame_number","identity_number","left","top","width","height","score","class","visibility"]
data=pd.read_csv("input/gt.txt",names=col_list)

car=data[data["class"]==3]
video_path=pathOut
cap=cv2.VideoCapture(video_path)

id1=29
numberOfImage=np.max(data["frame_number"])
fps=25
bound_box_list=[]

for i in range(numberOfImage-1):
    ret,frame=cap.read()
    if ret:
        frame=cv2.resize(frame,(960,540))
        filter_id1=np.logical_and(car["frame_number"]==i+1, car["identity_number"]==id1)
        if len(car[filter_id1])!=0:
            x=int(car[filter_id1].left.values[0]/2)
            y=int(car[filter_id1].top.values[0]/2)
            w=int(car[filter_id1].width.values[0]/2)
            h=int(car[filter_id1].height.values[0]/2)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.circle(frame,(int(x+w/2),int(y+h/2)),2,(0,0,255),-1)

            bound_box_list.append([i,x,y,w,h,int(x+w/2),int(y+h/2)])
            # frame , x , y , genişlik , center_x , center_y
            cv2.putText(frame,"Frame Number : "+str(i+1),(10,30),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,(0,0,255),2)
            cv2.imshow("frame",frame)
        if cv2.waitKey(1)&0xff==ord('q'):
            break
    else:
        print("okumadi")
        break
cap.release()
cv2.destroyAllWindows()

write_to_file(bound_box_list, "input/gt_new.txt") # bunu başka bir dersin sonunda ekledim sebebi ilerideki işlemde bunu kullanıyoruz

Bu kod, bir videodaki belirli bir nesneyi (bu durumda, bir arabanın kimliği) tespit etmek ve her çerçevede onun etrafına bir sınırlayıcı kutu çizmek için kullanılıyor. Aşağıda adım adım bu kodun nasıl çalıştığını açıklıyorum:

1. **Gerekli Kütüphanelerin İçe Aktarılması:**
   ```python
   import pandas as pd
   import numpy as np
   import cv2
   ```

2. **Dosya Yolu ve Veri Çerçevesinin Yüklenmesi:**
   ```python
   pathOut = "input/deneme.mp4"
   col_list = ["frame_number", "identity_number", "left", "top", "width", "height", "score", "class", "visibility"]
   data = pd.read_csv("input/gt.txt", names=col_list)
   ```
   - `pathOut` değişkeni, video dosyasının yolunu tutar.
   - `col_list`, veri çerçevesindeki sütun isimlerini tanımlar.
   - `data`, `gt.txt` dosyasından yüklenen verileri içerir.

3. **Belirli Bir Nesnenin (Bu Durumda Arabanın) Filtrelenmesi:**
   ```python
   car = data[data["class"] == 3]
   ```
   - `car`, sınıfı 3 (araba) olan tüm satırları içerir.

4. **Video Dosyasının Açılması:**
   ```python
   video_path = pathOut
   cap = cv2.VideoCapture(video_path)
   ```

5. **Gerekli Değişkenlerin Tanımlanması:**
   ```python
   id1 = 29
   numberOfImage = np.max(data["frame_number"])
   fps = 25
   bound_box_list = []
   ```
   - `id1`, takip edilecek nesnenin kimliğini belirtir.
   - `numberOfImage`, videodaki maksimum kare numarasını alır.
   - `fps`, videonun saniyedeki kare sayısını belirtir.
   - `bound_box_list`, sınırlayıcı kutu bilgilerini saklamak için bir liste olarak tanımlanır.

6. **Ana Döngü:**
   ```python
   for i in range(numberOfImage - 1):
       ret, frame = cap.read()
       if ret:
           frame = cv2.resize(frame, (960, 540))
           filter_id1 = np.logical_and(car["frame_number"] == i + 1, car["identity_number"] == id1)
           if len(car[filter_id1]) != 0:
               x = int(car[filter_id1].left.values[0] / 2)
               y = int(car[filter_id1].top.values[0] / 2)
               w = int(car[filter_id1].width.values[0] / 2)
               h = int(car[filter_id1].height.values[0] / 2)
               cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
               cv2.circle(frame, (int(x + w / 2), int(y + h / 2)), 2, (0, 0, 255), -1)
               bound_box_list.append([i, x, y, w, h, int(x + w / 2), int(y + h / 2)])
               cv2.putText(frame, "Frame Number : " + str(i + 1), (10, 30), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0, 0, 255), 2)
               cv2.imshow("frame", frame)
           if cv2.waitKey(1) & 0xff == ord('q'):
               break
       else:
           print("okumadi")
           break
   cap.release()
   cv2.destroyAllWindows()
   ```

   - `for i in range(numberOfImage - 1)` döngüsü, her kareyi işler.
   - `ret, frame = cap.read()` ile kare okunur.
   - `frame = cv2.resize(frame, (960, 540))` ile kare yeniden boyutlandırılır.
   - `filter_id1`, şu anki karedeki belirli kimliği (bu durumda 29) olan arabayı filtreler.
   - Eğer bu kimlikte bir araba varsa (`len(car[filter_id1]) != 0`),:
     - `x`, `y`, `w` ve `h` değerleri hesaplanır ve yarıya indirilir.
     - `cv2.rectangle` ile sınırlayıcı kutu çizilir.
     - `cv2.circle` ile merkez noktasına bir daire çizilir.
     - `bound_box_list` listesine sınırlayıcı kutu bilgileri eklenir.
     - `cv2.putText` ile kare numarası yazılır.
     - `cv2.imshow` ile kare görüntülenir.
   - `cv2.waitKey(1) & 0xff == ord('q')` koşulu, 'q' tuşuna basıldığında döngüyü sonlandırır.
   - Eğer kare okunamazsa (`else` bloğu), "okumadi" yazdırılır ve döngü sonlandırılır.
   - Döngü sonunda `cap.release()` ile video dosyası serbest bırakılır ve `cv2.destroyAllWindows()` ile tüm OpenCV pencereleri kapatılır.

Bu kod parçası, belirli bir nesneyi (kimliği bilinen bir arabayı) video boyunca takip etmek ve her karede bu nesnenin konumunu görselleştirmek için kullanılır.

# BOOSTING Takip Algoritması

In [2]:
# yukarıda bulunan 
import cv2
import pandas as pd
import time
import numpy as np

gt = pd.read_csv("input/gt_new.txt")
video_path = "input/deneme.mp4"
cap = cv2.VideoCapture(video_path)

# genel parametreler
initBB = None
fps = 25
frame_number = []
f = 0
success_track_frame_success = 0
track_list = []
start_time = time.time()

# tracker tanımlama (örneğin, KCF tracker)
tracker = cv2.TrackerKCF_create()

while True:
    time.sleep(0.01)
    ret, frame = cap.read()
    if ret:
        frame = cv2.resize(frame, (960, 540))
        (H, W) = frame.shape[:2]
        car_gt = gt[gt.frame_no == f]
        if len(car_gt) != 0:
            x = car_gt.x.values[0]
            y = car_gt.y.values[0]
            w = car_gt.w.values[0]
            h = car_gt.h.values[0]
            center_x = car_gt.center_x.values[0]
            center_y = car_gt.center_y.values[0]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.circle(frame, (center_x, center_y), 2, (0, 0, 255), -1)
            frame_number.append(f)
            f += 1
            # box
        if initBB is not None:
            (success, box) = tracker.update(frame)
            if f <= np.max(gt.frame_no):
                (x2, y2, w2, h2) = [int(i) for i in box]
                cv2.rectangle(frame, (x2, y2), (x2+w2, y2+h2), (0, 0, 255), 2)
                success_track_frame_success += 1
                track_center_x = int(x2 + w2 / 2)
                track_center_y = int(y2 + h2 / 2)
                track_list.append([f, track_center_x, track_center_y])

        if cv2.waitKey(1) & 0xff == ord('q'):
            break
        # şimdi benim video donsun ve yeni nesne seçebileyim onu takip edelim
        elif cv2.waitKey(1) & 0xff == ord('t'):
            initBB = cv2.selectROI("Frame", frame, fromCenter=False)
            tracker.init(frame, initBB)
        cv2.imshow("frame", frame)
    else:
        print("hata")
        break
cap.release()
cv2.destroyAllWindows()
stop = time.time()
time_diff = stop - start_time


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


## Tüm kod açıklama :
Tabii, kodun her bir parçasını ayrıntılı olarak açıklayabilirim. İşte adım adım açıklama:

### Gerekli Kütüphanelerin İçe Aktarılması

```python
import cv2
import pandas as pd
import time
import numpy as np
```
Bu kısımda OpenCV (cv2), pandas, time ve numpy kütüphaneleri içe aktarılıyor. OpenCV görüntü ve video işleme için, pandas veri analizi için, time zaman ölçümleri için ve numpy ise bilimsel hesaplamalar için kullanılıyor.

### Veri ve Video Yollarının Tanımlanması

```python
gt = pd.read_csv("input/gt_new.txt")
video_path = "input/deneme.mp4"
cap = cv2.VideoCapture(video_path)
```
- `gt`: Ground truth verileri içeren bir CSV dosyasını okur.
- `video_path`: Videonun yolunu belirler.
- `cap`: OpenCV kullanarak videoyu okumak için `VideoCapture` nesnesi oluşturur.

### Genel Parametrelerin Tanımlanması

```python
initBB = None
fps = 25
frame_number = []
f = 0
success_track_frame_success = 0
track_list = []
start_time = time.time()
```
- `initBB`: Nesnenin başlangıçta seçilen koordinatları için.
- `fps`: Videonun kare hızı (saniyedeki kare sayısı).
- `frame_number`: İşlenen kare numaralarını saklamak için bir liste.
- `f`: Geçerli kare numarası.
- `success_track_frame_success`: Başarılı takip edilen karelerin sayısı.
- `track_list`: Takip edilen nesnenin merkez koordinatlarını saklayan liste.
- `start_time`: Zaman ölçümü için başlangıç zamanı.

### Tracker Tanımlama

```python
tracker = cv2.TrackerKCF_create()
```
KCF (Kernelized Correlation Filters) tracker oluşturulur. Bu, nesne takibi için kullanılan bir algoritmadır.

### Ana Döngü

```python
while True:
    time.sleep(0.01)
    ret, frame = cap.read()
    if ret:
        frame = cv2.resize(frame, (960, 540))
        (H, W) = frame.shape[:2]
        car_gt = gt[gt.frame_no == f]
        if len(car_gt) != 0:
            x = car_gt.x.values[0]
            y = car_gt.y.values[0]
            w = car_gt.w.values[0]
            h = car_gt.h.values[0]
            center_x = car_gt.center_x.values[0]
            center_y = car_gt.center_y.values[0]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.circle(frame, (center_x, center_y), 2, (0, 0, 255), -1)
            frame_number.append(f)
            f += 1
        if initBB is not None:
            (success, box) = tracker.update(frame)
            if f <= np.max(gt.frame_no):
                (x2, y2, w2, h2) = [int(i) for i in box]
                cv2.rectangle(frame, (x2, y2), (x2+w2, y2+h2), (0, 0, 255), 2)
                success_track_frame_success += 1
                track_center_x = int(x2 + w2 / 2)
                track_center_y = int(y2 + h2 / 2)
                track_list.append([f, track_center_x, track_center_y])

        if cv2.waitKey(1) & 0xff == ord('q'):
            break
        elif cv2.waitKey(1) & 0xff == ord('t'):
            initBB = cv2.selectROI("Frame", frame, fromCenter=False)
            tracker.init(frame, initBB)
        cv2.imshow("frame", frame)
    else:
        print("hata")
        break
cap.release()
cv2.destroyAllWindows()
stop = time.time()
time_diff = stop - start_time
```

#### Döngü Açıklaması

1. **Video Karelerini Okuma ve Yeniden Boyutlandırma:**
   ```python
   time.sleep(0.01)
   ret, frame = cap.read()
   if ret:
       frame = cv2.resize(frame, (960, 540))
       (H, W) = frame.shape[:2]
   ```
   - Her döngüde 0.01 saniye beklenir.
   - `cap.read()` ile video karesi okunur.
   - Kare başarıyla okunursa (`ret` True ise), kare 960x540 boyutlarına yeniden boyutlandırılır ve boyutları alınır.

2. **Ground Truth Verilerini Kullanma:**
   ```python
   car_gt = gt[gt.frame_no == f]
   if len(car_gt) != 0:
       x = car_gt.x.values[0]
       y = car_gt.y.values[0]
       w = car_gt.w.values[0]
       h = car_gt.h.values[0]
       center_x = car_gt.center_x.values[0]
       center_y = car_gt.center_y.values[0]
       cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
       cv2.circle(frame, (center_x, center_y), 2, (0, 0, 255), -1)
       frame_number.append(f)
       f += 1
   ```
   - Ground truth verileri geçerli kare numarası ile filtrelenir.
   - Eğer bu kare için ground truth verisi varsa, koordinatlar alınır ve kare üzerine yeşil bir dikdörtgen ve kırmızı bir daire çizilir.

3. **Nesne Takibi:**
   ```python
   if initBB is not None:
       (success, box) = tracker.update(frame)
       if f <= np.max(gt.frame_no):
           (x2, y2, w2, h2) = [int(i) for i in box]
           cv2.rectangle(frame, (x2, y2), (x2+w2, y2+h2), (0, 0, 255), 2)
           success_track_frame_success += 1
           track_center_x = int(x2 + w2 / 2)
           track_center_y = int(y2 + h2 / 2)
           track_list.append([f, track_center_x, track_center_y])
   ```
   - Eğer `initBB` tanımlanmışsa (yani bir nesne seçilmişse), tracker kareyi günceller.
   - Güncelleme başarılı olursa, yeni koordinatlar alınır ve kare üzerine kırmızı bir dikdörtgen çizilir. Ayrıca, takip edilen nesnenin merkez koordinatları listeye eklenir.

4. **Klavye Girdileri:**
   ```python
   if cv2.waitKey(1) & 0xff == ord('q'):
       break
   elif cv2.waitKey(1) & 0xff == ord('t'):
       initBB = cv2.selectROI("Frame", frame, fromCenter=False)
       tracker.init(frame, initBB)
   cv2.imshow("frame", frame)
   ```
   - `q` tuşuna basıldığında döngü kırılır ve işlem sonlanır.
   - `t` tuşuna basıldığında, kullanıcı yeni bir nesne seçebilir ve tracker bu nesneyi takip etmeye başlar.
   - Kare gösterilir.

5. **Döngü Sonu:**
   ```python
   else:
       print("hata")
       break
   cap.release()
   cv2.destroyAllWindows()
   stop = time.time()
   time_diff = stop - start_time
   ```
   - Eğer kare okunamazsa hata mesajı yazdırılır ve döngü kırılır.
   - Video serbest bırakılır ve tüm pencereler kapatılır.
   - İşlem süresi hesaplanır.

Bu kod, verilen ground truth verileri ve videoyu kullanarak nesne takibi yapar. Kullanıcı, `t` tuşuna basarak yeni nesneler seçebilir ve takip edebilir.